In [ ]:
from ssjlatbin.io import read_config
config = read_config('../scripts/default_config.toml')

In [ ]:
print(config)

In [ ]:
import datetime
from ssjlatbin.cdf import get_orbit_numbered_ssj_range_dataframe

dmsp_number = 13
sdt = datetime.datetime(2006,12,2)
edt = datetime.datetime(2006,12,20)

config['latbin']['delta_lat']=3
config['io']['ssj_cdf_root_dir']='../dat' #Change path to be appropriate
config['calculation']['uncertainty_tolerance']=75
df = get_orbit_numbered_ssj_range_dataframe(dmsp_number,sdt,edt,config)

In [ ]:
import pandas as pd

In [ ]:
from ssjlatbin.latbin_pandas import bin_by_latitude

In [ ]:
lbdf = bin_by_latitude(df,config)

In [ ]:
lbdf

In [ ]:
from ssjlatbin.io import dataframe_to_latbinned_flux

In [ ]:
t,lats,flag,fluxes = dataframe_to_latbinned_flux(lbdf,'ion_soft_number')

In [ ]:
from ssjlatbin.fluxcalculations import average_particle_energy
t,lats,_,influxs = dataframe_to_latbinned_flux(lbdf,'ion_soft_number')
t,lats,_,eefluxh = dataframe_to_latbinned_flux(lbdf,'ele_hard_energy')
t,lats,_,eeflux = dataframe_to_latbinned_flux(lbdf,'ele_total_energy')
t,lats,_,enflux = dataframe_to_latbinned_flux(lbdf,'ele_total_number')
t,lats,_,sza = dataframe_to_latbinned_flux(lbdf,'solar_zenith_angle')

eavg = average_particle_energy(eeflux,enflux)
eavg[enflux==0]=0 #Prevent NaN values if zero number flux

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

def latlabel(value,tick_number):
    ilat = int(np.round(value))
    if ilat>=len(lats):
        ilat=len(lats)-1
    return '{}'.format(lats[ilat])

N = lats > 0

f = plt.figure(figsize=(8,6),dpi=150)
ax = f.add_subplot(111)
mappable = ax.pcolormesh(t,np.arange(lats.size),influxs.T,vmax=1e12,norm=LogNorm(),cmap='Reds')
ax.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
f.colorbar(mappable,label='Soft Ion Number Flux \n [particles/m^2]')
f.autofmt_xdate()

# f = plt.figure(figsize=(8,6),dpi=150)
# ax = f.add_subplot(111)
# mappable = ax.pcolormesh(t,np.arange(lats.size),enflux.T,vmax=1e13,norm=LogNorm())
# ax.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
# f.colorbar(mappable,label='Total Electron Number Flux \n [particles/m^2]')
# f.autofmt_xdate()

# f = plt.figure(figsize=(8,6),dpi=150)
# ax = f.add_subplot(111)
# mappable = ax.pcolormesh(t,np.arange(lats.size),eeflux.T,vmin=.0001,vmax=10,norm=LogNorm())
# ax.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
# f.colorbar(mappable,label='Total Electron Energy Flux \n [mW/m^2]')
# f.autofmt_xdate()

f3 = plt.figure(figsize=(8,6),dpi=150)
ax3 = f3.add_subplot(111)
mappable2 = ax3.pcolormesh(t,np.arange(lats.size),influxs.T,vmax=1e12,norm=LogNorm(),cmap='Reds')
mappable = ax3.pcolormesh(t,np.arange(lats.size),eefluxh.T,vmin=.01,vmax=10,norm=LogNorm())
ax3.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
f3.colorbar(mappable,label='Hard Electron Energy Flux \n [mW/m^2]')
f3.autofmt_xdate()

f4 = plt.figure(figsize=(8,6),dpi=150)
ax4 = f4.add_subplot(111)
mappable = ax4.pcolormesh(t,np.arange(lats.size),eavg.T,cmap='plasma',vmin=100,vmax=10000)
ax4.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
f4.colorbar(mappable,label='Electron Average Energy \n [eV]')
f4.autofmt_xdate()

f5 = plt.figure(figsize=(8,6),dpi=150)
ax5 = f5.add_subplot(111)
mappable = ax5.pcolormesh(t,np.arange(lats.size),np.degrees(sza.T),cmap='plasma_r')
ax5.yaxis.set_major_formatter(plt.FuncFormatter(latlabel))
f5.colorbar(mappable,label='Solar Zenith Angle [deg]')
f5.autofmt_xdate()

# for thisax in [ax,ax3,ax4]:
#     thisax.set_xlim([datetime.datetime(2000,7,14),datetime.datetime(2000,7,19)])